In [134]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import sklearn.datasets
import altair as alt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None

In [2]:
iris = sklearn.datasets.load_iris(as_frame=True)
irisX = iris.data
irisy = iris.target
setA = irisy <= 1
setB = irisy > 0
setC = irisy != 1

irisAX = irisX[setA]
irisAy = irisy[setA]
irisBX = irisX[setB]
irisBy = irisy[setB] - 1
irisCX = irisX[setC]
irisCy = irisy[setC]/2

trainXs = [irisAX,irisBX,irisCX]
trainys = [irisAy,irisBy,irisCy]

In [3]:
trainX, testX, trainy, testy = train_test_split(
    irisAX,
    irisAy,
    test_size=0.2)

In [4]:
# creates a random unit vector of ndim dimensions
def ruvect(ndim):
    out=[]
    for i in range(ndim):
        out.append(np.random.randint(-100,100))
    out = np.array(out)
    out = out/np.linalg.norm(out)
    return out

In [5]:
# finds centroid of list of points in np array of nxd dimensions, returns point of d dims
def calc_centroid(cluster):
    return np.mean(cluster, axis=0)

In [181]:
# X and y expected to be numpy arrays where y only contains 0's and 1's
def lda(X,y):
    # 1 is for intercept L(w) = w0 + w1x1 + w2x2 +...+ wdxd
    w = ruvect(X.shape[1]) 
    
    # adds a column of 1's to front (for intercept) -- unecessary as position of w doesnt matter
#     X = np.append(np.array([[1] * X.shape[0]]).T, X, axis=1)
    X0 = X[y==0]
    X1 = X[y==1]
    
    # Centroids of data (d dimensional)
    mu0 = calc_centroid(X0)
    mu1 = calc_centroid(X1)
    
    B = np.einsum("i,j->ij", mu0-mu1, mu0-mu1)
    S = np.einsum("Nd,Ni->di", X0-mu0, X0-mu0)
    eta = 0.1
    djw_norm = 100
    iters = 0
    while djw_norm > 0.01 and iters<500:
        # Derivative of J w
        djnum1 = 2*np.matmul(B,w)*(np.matmul(np.matmul(w,S),w))
        djnum2 = 2*np.matmul(S,w)*(np.matmul(np.matmul(w,B),w))
        djdem = np.matmul(np.matmul(w,S),w)**2
        
        djw = (djnum1-djnum2)/djdem
        w = w + eta*djw
        djw_norm = np.linalg.norm(djw)
#         print(djw_norm, w)
        iters+=1
    
    return w,(np.dot(w,mu0)+np.dot(w,mu1))/2

def lda_pred(w,sep,X):
    projs=np.matmul(w,X.T)
    return projs>sep

50     True
51     True
52     True
53     True
54     True
       ... 
145    True
146    True
147    True
148    True
149    True
Length: 100, dtype: bool

In [7]:
def logreg(X,y):
    w = ruvect(X.shape[1]+1)
    X = np.append(np.array([[1] * X.shape[0]]).T, X, axis=1) # add intercept col

    eta=0.1
    errmag = 100
    iters=0
    while errmag> 0.1 and iters<100:
        Xw = np.einsum("Nd,d->N",X,w)
        phat = np.divide(1,1+np.exp(-Xw))
        derror = np.einsum("N,Nd->d",y-phat,X)
        
        w += eta*derror
        errmag = np.linalg.norm(derror)
        iters+=1
    return w

def logreg_pred(X,w):
    X = np.append(np.array([[1] * X.shape[0]]).T, X, axis=1) # add intercept column
    return 1.0/(1-np.exp(np.einsum("Nd,d->N",X,w))) < 0.5

# Comparison

In [182]:
w,sep = lda(irisBX,irisBy)
lda_pred(w,sep,irisBX) == irisBy

lda_ws=[]
lda_seps=[]
for x,y in zip(trainXs,trainys):
    w,sep = lda(x,y)
    lda_ws.append(w)
    lda_seps.append(sep)
lda_ws=np.asarray(lda_ws)
lda_ws

array([[ 0.17938494, -0.46674436,  1.87804537,  1.36649524],
       [-0.04632363, -0.33800016,  0.06250817,  0.95664025],
       [-0.19162776,  0.57213847, -2.62864153, -2.40254213]])

In [9]:
lda_sk = LinearDiscriminantAnalysis(solver="lsqr")
lda_sk_ws=[]
for X,y in zip(trainXs,trainys):
    lda_sk.fit(X=X,y=y)
    lda_sk_ws.append(lda_sk.coef_)
# lda_pred(trainX,lda_sk_w,lda_sep) == trainy
lda_sk_ws = np.asarray(lda_sk_ws)[:,0]
lda_sk_ws

array([[ -3.11507166, -18.39077486,  22.21040275,  31.4736377 ],
       [ -3.6288803 ,  -5.69247004,   7.11237519,  12.6388175 ],
       [-16.16463467, -12.26541407,  37.26365575,  37.50814735]])

In [10]:
np.einsum("ij,ij->ij",lda_ws,1/np.array(lda_sk_ws))

array([[-0.05689756,  0.02520314,  0.0838352 ,  0.0430589 ],
       [ 0.01280047,  0.05922321,  0.00884183,  0.07543849],
       [-0.01188953,  0.04657019,  0.07051618,  0.06403047]])

## Resulting ws are quite different, but somewhat similar relative to eachother

In [183]:
for X,y,w,sep in zip(trainXs, trainys, lda_ws, lda_seps):
    print((lda_pred(X,w,sep) == y).describe().freq/100)
lda_pred(trainXs[0],lda_ws[0],lda_seps[0])

IndexError: tuple index out of range

In [ ]:
logreg_sk = LogisticRegression(solver = "sag")
logreg_sk.fit(X=trainX, y=trainy)
logreg_sk.coef_